In [1]:
import sys

sys.path.append("../")

import pennylane as qml
import pennylane.numpy as np
from main import *

from qiskit_aer.primitives import Sampler 

# np.random.seed(0)  # For reproducibility

In [2]:
num_wires = 2

In [3]:
num_layers = 2

var_state_class = RY_ansatz(num_wires, num_layers)
var_state = var_state_class.construct_circuit
num_params = var_state_class.num_params

params = np.random.randn(num_params) * 0.01

In [4]:
X, P = X_and_P_ops(range(num_wires), 
                    x_min=-5, 
                    x_max=5, 
                    semiclassical=True)

Q = [X.pow(2).s_prod(0.5), P.pow(2).s_prod(0.5)]

# Pennylane

In [5]:
device = qml.device( 'lightning.qubit', wires=4*num_wires, shots=100 ) 

In [6]:
expval_XP( params, var_state, Q, device )

12.655446269317157

In [7]:
grad_XP( params, var_state, Q, device )

tensor([-0.11436404,  0.00396422,  0.43774343, -0.44159056], requires_grad=True)

In [8]:
varqft = VarFourier( Q, device, var_state=var_state )

In [9]:
params_ground, energies_ground = varqft.run(
    params,
    max_iterations=500,
    learning_rate=0.1,
)

Step:    500, Energy:     1.453288


# Qiskit

In [10]:
sampler = Sampler()

In [11]:
expval_XP( params, var_state, Q, sampler, shots=100 )

12.673211557239119

In [12]:
grad_XP( params, var_state, Q, sampler )

tensor([-0.11070776,  0.05742379, -0.00697768, -0.21020733], requires_grad=True)

In [13]:
varqft = VarFourier( Q, sampler, var_state=var_state )

In [14]:
params_ground, energies_ground = varqft.run(
    params,
    max_iterations=500,
    learning_rate=0.1,
)

Step:    500, Energy:     1.445056
